In [ ]:
!pip install textract
!pip install transformers
!pip install peft


In [1]:
from transformers import AutoTokenizer
import os
import pandas as pd
import textract
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig
import requests
import zipfile

peft_model_id = "smangrul/twitter_complaints_bigscience_T0_3B_LORA_SEQ_2_SEQ_LM"
config = PeftConfig.from_pretrained(peft_model_id)

ModuleNotFoundError: No module named 'textract'

In [ ]:
def download_and_unzip(url, folder_name):
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Download the file
    response = requests.get(url)
    zip_path = os.path.join(folder_name, 'temp.zip')
    with open(zip_path, 'wb') as file:
        file.write(response.content)

    # Unzip the file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(folder_name)

    # Remove the zip file
    os.remove(zip_path)

In [ ]:
zip_path = "https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/Archive.zip"
unzip_folder = "train_data"

download_and_unzip(zip_path, unzip_folder)

# Fine Tune

In [ ]:
def read_file(file_path):
    _, ext = os.path.splitext(file_path)
    if ext in ['.txt', '.csv']:
        with open(file_path, 'r') as file:
            return file.read()
    elif ext in ['.docx', '.pdf']:
        return textract.process(file_path).decode()

def create_dataset(folder_path, tokenizer_name):
    texts = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            text = read_file(file_path)
            if text:
                texts.append(text)
    dataset = Dataset.from_pandas(pd.DataFrame(texts, columns=['text']))
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    tokenized_dataset = dataset.map(lambda x: tokenizer(x['text']), batched=True)
    return tokenized_dataset

In [ ]:
dataset = create_dataset(unzip_folder, config.base_model_name_or_path)
for i, item in enumerate(dataset):
    if i == 0:
        print(item)
        break